In [ ]:
import pandas as pd 
import numpy as np
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense,Conv2D,LSTM,RNN,MaxPooling2D,Flatten,BatchNormalization,Dropout
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data=pd.read_csv("/kaggle/input/air-quality-data-in-india/city_hour.csv")
data=pd.DataFrame(data)

In [7]:
from sklearn.impute import SimpleImputer


numeric_columns = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 
                   'Benzene', 'Toluene', 'Xylene', 'AQI']


categorical_columns = ['AQI_Bucket']

# Imputer for numeric columns
imputer_numeric = SimpleImputer(strategy='mean')
data[numeric_columns] = imputer_numeric.fit_transform(data[numeric_columns])

# Imputer for categorical columns
imputer_categorical = SimpleImputer(strategy='most_frequent')
data[categorical_columns] = imputer_categorical.fit_transform(data[categorical_columns]) 

In [8]:
data.head(30)

,City,Datetime,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01 01:00:00,67.622994,119.075804,1.00,40.01,36.37,23.607959,1.00,122.07,34.798979,0.0,0.00,0.0,166.4135,Moderate
1,Ahmedabad,2015-01-01 02:00:00,67.622994,119.075804,0.02,27.75,19.73,23.607959,0.02,85.90,34.798979,0.0,0.00,0.0,166.4135,Moderate
2,Ahmedabad,2015-01-01 03:00:00,67.622994,119.075804,0.08,19.32,11.08,23.607959,0.08,52.83,34.798979,0.0,0.00,0.0,166.4135,Moderate
3,Ahmedabad,2015-01-01 04:00:00,67.622994,119.075804,0.30,16.45,9.20,23.607959,0.30,39.53,153.580000,0.0,0.00,0.0,166.4135,Moderate
4,Ahmedabad,2015-01-01 05:00:00,67.622994,119.075804,0.12,14.90,7.85,23.607959,0.12,32.63,34.798979,0.0,0.00,0.0,166.4135,Moderate
5,Ahmedabad,2015-01-01 06:00:00,67.622994,119.075804,0.33,15.95,10.82,23.607959,0.33,29.87,64.250000,0.0,0.00,0.0,166.4135,Moderate
6,Ahmedabad,2015-01-01 07:00:00,67.622994,119.075804,0.45,15.94,12.47,23.607959,0.45,27.41,191.960000,0.0,0.00,0.0,166.4135,Moderate
7,Ahmedabad,2015-01-01 08:00:00,67.622994,119.075804,1.03,16.66,16.48,23.607959,1.03,20.92,177.210000,0.0,0.00,0.0,166.4135,Moderate
8,Ahmedabad,2015-01-01 09:00:00,67.622994,119.075804,1.47,16.25,18.02,23.607959,1.47,16.45,122.080000,0.0,0.00,0.0,166.4135,Moderate
9,Ahmedabad,2015-01-01 10:00:00,67.622994,119.075804,2.05,13.78,16.08,23.607959,2.05,15.14,34.798979,0.0,0.00,0.0,166.4135,Moderate


In [86]:
df = pd.DataFrame(data)

# Drop duplicates based on 'StationId' and 'City'
unique_station_city_df = df[['StationId', 'StationName']].drop_duplicates()
unique_station_city_df.to_csv("station")

The index of "Secretariat, Amaravati - APPCB" is: 0


In [87]:
stations = [
{'Unnamed: 0': 0,
  'StationId': 'AP001',
  'StationName': 'Secretariat, Amaravati - APPCB'},
 {'Unnamed: 0': 1,
  'StationId': 'AP002',
  'StationName': 'Anand Kala Kshetram, Rajamahendravaram - APPCB'},
 {'Unnamed: 0': 2,
  'StationId': 'AP003',
  'StationName': 'Tirumala, Tirupati - APPCB'},
 {'Unnamed: 0': 3,
  'StationId': 'AP004',
  'StationName': 'PWD Grounds, Vijayawada - APPCB'},
 {'Unnamed: 0': 4,
  'StationId': 'AP005',
  'StationName': 'GVM Corporation, Visakhapatnam - APPCB'},
 {'Unnamed: 0': 5,
  'StationId': 'AS001',
  'StationName': 'Railway Colony, Guwahati - APCB'},
 {'Unnamed: 0': 6,
  'StationId': 'BR001',
  'StationName': 'Collectorate, Gaya - BSPCB'},
 {'Unnamed: 0': 7,
  'StationId': 'BR002',
  'StationName': 'SFTI Kusdihra, Gaya - BSPCB'},
 {'Unnamed: 0': 8,
  'StationId': 'BR003',
  'StationName': 'Industrial Area, Hajipur - BSPCB'},
 {'Unnamed: 0': 9,
  'StationId': 'BR004',
  'StationName': 'Muzaffarpur Collectorate, Muzaffarpur - BSPCB'},
 {'Unnamed: 0': 10,
  'StationId': 'BR005',
  'StationName': 'DRM Office Danapur, Patna - BSPCB'},
 {'Unnamed: 0': 11,
  'StationId': 'BR006',
  'StationName': 'Govt. High School Shikarpur, Patna - BSPCB'},
 {'Unnamed: 0': 12,
  'StationId': 'BR007',
  'StationName': 'IGSC Planetarium Complex, Patna - BSPCB'},
 {'Unnamed: 0': 13,
  'StationId': 'BR008',
  'StationName': 'Muradpur, Patna - BSPCB'},
 {'Unnamed: 0': 14,
  'StationId': 'BR009',
  'StationName': 'Rajbansi Nagar, Patna - BSPCB'},
 {'Unnamed: 0': 15,
  'StationId': 'BR010',
  'StationName': 'Samanpura, Patna - BSPCB'},
 {'Unnamed: 0': 16,
  'StationId': 'CH001',
  'StationName': 'Sector-25, Chandigarh - CPCC'},
 {'Unnamed: 0': 17,
  'StationId': 'DL001',
  'StationName': 'Alipur, Delhi - DPCC'},
 {'Unnamed: 0': 18,
  'StationId': 'DL002',
  'StationName': 'Anand Vihar, Delhi - DPCC'},
 {'Unnamed: 0': 19,
  'StationId': 'DL003',
  'StationName': 'Ashok Vihar, Delhi - DPCC'},
 {'Unnamed: 0': 20,
  'StationId': 'DL004',
  'StationName': 'Aya Nagar, Delhi - IMD'},
 {'Unnamed: 0': 21,
  'StationId': 'DL005',
  'StationName': 'Bawana, Delhi - DPCC'},
 {'Unnamed: 0': 22,
  'StationId': 'DL006',
  'StationName': 'Burari Crossing, Delhi - IMD'},
 {'Unnamed: 0': 23,
  'StationId': 'DL007',
  'StationName': 'CRRI Mathura Road, Delhi - IMD'},
 {'Unnamed: 0': 24, 'StationId': 'DL008', 'StationName': 'DTU, Delhi - CPCB'},
 {'Unnamed: 0': 25,
  'StationId': 'DL009',
  'StationName': 'Dr. Karni Singh Shooting Range, Delhi - DPCC'},
 {'Unnamed: 0': 26,
  'StationId': 'DL010',
  'StationName': 'Dwarka-Sector 8, Delhi - DPCC'},
 {'Unnamed: 0': 27,
  'StationId': 'DL011',
  'StationName': 'East Arjun Nagar, Delhi - CPCB'},
 {'Unnamed: 0': 28,
  'StationId': 'DL012',
  'StationName': 'IGI Airport (T3), Delhi - IMD'},
 {'Unnamed: 0': 29,
  'StationId': 'DL013',
  'StationName': 'IHBAS, Dilshad Garden, Delhi - CPCB'},
 {'Unnamed: 0': 30, 'StationId': 'DL014', 'StationName': 'ITO, Delhi - CPCB'},
 {'Unnamed: 0': 31,
  'StationId': 'DL015',
  'StationName': 'Jahangirpuri, Delhi - DPCC'},
 {'Unnamed: 0': 32,
  'StationId': 'DL016',
  'StationName': 'Jawaharlal Nehru Stadium, Delhi - DPCC'},
 {'Unnamed: 0': 33,
  'StationId': 'DL017',
  'StationName': 'Lodhi Road, Delhi - IMD'},
 {'Unnamed: 0': 34,
  'StationId': 'DL018',
  'StationName': 'Major Dhyan Chand National Stadium, Delhi - DPCC'},
 {'Unnamed: 0': 35,
  'StationId': 'DL019',
  'StationName': 'Mandir Marg, Delhi - DPCC'},
 {'Unnamed: 0': 36,
  'StationId': 'DL020',
  'StationName': 'Mundka, Delhi - DPCC'},
 {'Unnamed: 0': 37,
  'StationId': 'DL021',
  'StationName': 'NSIT Dwarka, Delhi - CPCB'},
 {'Unnamed: 0': 38,
  'StationId': 'DL022',
  'StationName': 'Najafgarh, Delhi - DPCC'},
 {'Unnamed: 0': 39,
  'StationId': 'DL023',
  'StationName': 'Narela, Delhi - DPCC'},
 {'Unnamed: 0': 40,
  'StationId': 'DL024',
  'StationName': 'Nehru Nagar, Delhi - DPCC'},
 {'Unnamed: 0': 41,
  'StationId': 'DL025',
  'StationName': 'North Campus, DU, Delhi - IMD'},
 {'Unnamed: 0': 42,
  'StationId': 'DL026',
  'StationName': 'Okhla Phase-2, Delhi - DPCC'},
 {'Unnamed: 0': 43,
  'StationId': 'DL027',
  'StationName': 'Patparganj, Delhi - DPCC'},
 {'Unnamed: 0': 44,
  'StationId': 'DL028',
  'StationName': 'Punjabi Bagh, Delhi - DPCC'},
 {'Unnamed: 0': 45, 'StationId': 'DL029', 'StationName': 'Pusa, Delhi - DPCC'},
 {'Unnamed: 0': 46, 'StationId': 'DL030', 'StationName': 'Pusa, Delhi - IMD'},
 {'Unnamed: 0': 47,
  'StationId': 'DL031',
  'StationName': 'R K Puram, Delhi - DPCC'},
 {'Unnamed: 0': 48,
  'StationId': 'DL032',
  'StationName': 'Rohini, Delhi - DPCC'},
 {'Unnamed: 0': 49,
  'StationId': 'DL033',
  'StationName': 'Shadipur, Delhi - CPCB'},
 {'Unnamed: 0': 50,
  'StationId': 'DL034',
  'StationName': 'Sirifort, Delhi - CPCB'},
 {'Unnamed: 0': 51,
  'StationId': 'DL035',
  'StationName': 'Sonia Vihar, Delhi - DPCC'},
 {'Unnamed: 0': 52,
  'StationId': 'DL036',
  'StationName': 'Sri Aurobindo Marg, Delhi - DPCC'},
 {'Unnamed: 0': 53,
  'StationId': 'DL037',
  'StationName': 'Vivek Vihar, Delhi - DPCC'},
 {'Unnamed: 0': 54,
  'StationId': 'DL038',
  'StationName': 'Wazirpur, Delhi - DPCC'},
 {'Unnamed: 0': 55,
  'StationId': 'GJ001',
  'StationName': 'Maninagar, Ahmedabad - GPCB'},
 {'Unnamed: 0': 56,
  'StationId': 'GJ002',
  'StationName': 'GIDC, Ankleshwar - GPCB'},
 {'Unnamed: 0': 57,
  'StationId': 'GJ003',
  'StationName': 'Sector-10, Gandhinagar - GPCB'},
 {'Unnamed: 0': 58,
  'StationId': 'GJ004',
  'StationName': 'GIDC, Nandesari - Nandesari Ind. Association'},
 {'Unnamed: 0': 59,
  'StationId': 'GJ005',
  'StationName': 'Phase-1 GIDC, Vapi - GPCB'},
 {'Unnamed: 0': 60,
  'StationId': 'GJ006',
  'StationName': 'Phase-4 GIDC, Vatva - GPCB'},
 {'Unnamed: 0': 61,
  'StationId': 'HR001',
  'StationName': 'Patti Mehar, Ambala - HSPCB'},
 {'Unnamed: 0': 62,
  'StationId': 'HR002',
  'StationName': 'Arya Nagar, Bahadurgarh - HSPCB'},
 {'Unnamed: 0': 63,
  'StationId': 'HR003',
  'StationName': 'Nathu Colony, Ballabgarh - HSPCB'},
 {'Unnamed: 0': 64,
  'StationId': 'HR004',
  'StationName': 'H.B. Colony, Bhiwani - HSPCB'},
 {'Unnamed: 0': 65,
  'StationId': 'HR005',
  'StationName': 'Municipal Corporation Office, Dharuhera - HSPCB'},
 {'Unnamed: 0': 66,
  'StationId': 'HR006',
  'StationName': 'New Industrial Town, Faridabad - HSPCB'},
 {'Unnamed: 0': 67,
  'StationId': 'HR007',
  'StationName': 'Sector 11, Faridabad - HSPCB'},
 {'Unnamed: 0': 68,
  'StationId': 'HR008',
  'StationName': 'Sector 30, Faridabad - HSPCB'},
 {'Unnamed: 0': 69,
  'StationId': 'HR009',
  'StationName': 'Sector- 16A, Faridabad - HSPCB'},
 {'Unnamed: 0': 70,
  'StationId': 'HR010',
  'StationName': 'Huda Sector, Fatehabad - HSPCB'},
 {'Unnamed: 0': 71,
  'StationId': 'HR011',
  'StationName': 'NISE Gwal Pahari, Gurugram - IMD'},
 {'Unnamed: 0': 72,
  'StationId': 'HR012',
  'StationName': 'Sector-51, Gurugram - HSPCB'},
 {'Unnamed: 0': 73,
  'StationId': 'HR013',
  'StationName': 'Teri Gram, Gurugram - HSPCB'},
 {'Unnamed: 0': 74,
  'StationId': 'HR014',
  'StationName': 'Vikas Sadan, Gurugram - HSPCB'},
 {'Unnamed: 0': 75,
  'StationId': 'HR015',
  'StationName': 'Urban Estate-II, Hisar - HSPCB'},
 {'Unnamed: 0': 76,
  'StationId': 'HR016',
  'StationName': 'Police Lines, Jind - HSPCB'},
 {'Unnamed: 0': 77,
  'StationId': 'HR017',
  'StationName': 'Rishi Nagar, Kaithal - HSPCB'},
 {'Unnamed: 0': 78,
  'StationId': 'HR018',
  'StationName': 'Sector-12, Karnal - HSPCB'},
 {'Unnamed: 0': 79,
  'StationId': 'HR019',
  'StationName': 'Sector-7, Kurukshetra - HSPCB'},
 {'Unnamed: 0': 80,
  'StationId': 'HR020',
  'StationName': 'General Hospital, Mandikhera - HSPCB'},
 {'Unnamed: 0': 81,
  'StationId': 'HR021',
  'StationName': 'Sector-2 IMT, Manesar - HSPCB'},
 {'Unnamed: 0': 82,
  'StationId': 'HR022',
  'StationName': 'Shastri Nagar, Narnaul - HSPCB'},
 {'Unnamed: 0': 83,
  'StationId': 'HR023',
  'StationName': 'Shyam Nagar, Palwal - HSPCB'},
 {'Unnamed: 0': 84,
  'StationId': 'HR024',
  'StationName': 'Sector-6, Panchkula - HSPCB'},
 {'Unnamed: 0': 85,
  'StationId': 'HR025',
  'StationName': 'Sector-18, Panipat - HSPCB'},
 {'Unnamed: 0': 86,
  'StationId': 'HR026',
  'StationName': 'MD University, Rohtak - HSPCB'},
 {'Unnamed: 0': 87,
  'StationId': 'HR027',
  'StationName': 'F-Block, Sirsa - HSPCB'},
 {'Unnamed: 0': 88,
  'StationId': 'HR028',
  'StationName': 'Murthal, Sonipat - HSPCB'},
 {'Unnamed: 0': 89,
  'StationId': 'HR029',
  'StationName': 'Gobind Pura, Yamuna Nagar - HSPCB'},
 {'Unnamed: 0': 90,
  'StationId': 'JH001',
  'StationName': 'Tata Stadium, Jorapokhar - JSPCB'},
 {'Unnamed: 0': 91,
  'StationId': 'KA001',
  'StationName': 'Vidayagiri, Bagalkot - KSPCB'},
 {'Unnamed: 0': 92,
  'StationId': 'KA002',
  'StationName': 'BTM Layout, Bengaluru - CPCB'},
 {'Unnamed: 0': 93,
  'StationId': 'KA003',
  'StationName': 'BWSSB Kadabesanahalli, Bengaluru - CPCB'},
 {'Unnamed: 0': 94,
  'StationId': 'KA004',
  'StationName': 'Bapuji Nagar, Bengaluru - KSPCB'},
 {'Unnamed: 0': 95,
  'StationId': 'KA005',
  'StationName': 'City Railway Station, Bengaluru - KSPCB'},
 {'Unnamed: 0': 96,
  'StationId': 'KA006',
  'StationName': 'Hebbal, Bengaluru - KSPCB'},
 {'Unnamed: 0': 97,
  'StationId': 'KA007',
  'StationName': 'Hombegowda Nagar, Bengaluru - KSPCB'},
 {'Unnamed: 0': 98,
  'StationId': 'KA008',
  'StationName': 'Jayanagar 5th Block, Bengaluru - KSPCB'},
 {'Unnamed: 0': 99,
  'StationId': 'KA009',
  'StationName': 'Peenya, Bengaluru - CPCB'},
 {'Unnamed: 0': 100,
  'StationId': 'KA010',
  'StationName': 'Sanegurava Halli, Bengaluru - KSPCB'},
 {'Unnamed: 0': 101,
  'StationId': 'KA011',
  'StationName': 'Silk Board, Bengaluru - KSPCB'},
 {'Unnamed: 0': 102,
  'StationId': 'KA012',
  'StationName': 'Urban, Chamarajanagar - KSPCB'},
 {'Unnamed: 0': 103,
  'StationId': 'KA013',
  'StationName': 'Chikkaballapur Rural, Chikkaballapur - KSPCB'},
 {'Unnamed: 0': 104,
  'StationId': 'KA014',
  'StationName': 'Kalyana Nagara, Chikkamagaluru - KSPCB'},
 {'Unnamed: 0': 105,
  'StationId': 'KA015',
  'StationName': 'Deshpande Nagar, Hubballi - KSPCB'},
 {'Unnamed: 0': 106,
  'StationId': 'KA016',
  'StationName': 'Lal Bahadur Shastri Nagar, Kalaburagi - KSPCB'},
 {'Unnamed: 0': 107,
  'StationId': 'KA017',
  'StationName': 'Hebbal 1st Stage, Mysuru - KSPCB'},
 {'Unnamed: 0': 108,
  'StationId': 'KA018',
  'StationName': 'Vijay Nagar, Ramanagara - KSPCB'},
 {'Unnamed: 0': 109,
  'StationId': 'KA019',
  'StationName': 'Ibrahimpur, Vijayapura - KSPCB'},
 {'Unnamed: 0': 110,
  'StationId': 'KA020',
  'StationName': 'Collector Office, Yadgir - KSPCB'},
 {'Unnamed: 0': 111,
  'StationId': 'KL001',
  'StationName': 'Udyogamandal, Eloor - Kerala PCB'},
 {'Unnamed: 0': 112,
  'StationId': 'KL002',
  'StationName': 'Kacheripady, Ernakulam - Kerala PCB'},
 {'Unnamed: 0': 113,
  'StationId': 'KL003',
  'StationName': 'Thavakkara, Kannur - Kerala PCB'},
 {'Unnamed: 0': 114,
  'StationId': 'KL004',
  'StationName': 'Vyttila, Kochi - Kerala PCB'},
 {'Unnamed: 0': 115,
  'StationId': 'KL005',
  'StationName': 'Polayathode, Kollam - Kerala PCB'},
 {'Unnamed: 0': 116,
  'StationId': 'KL006',
  'StationName': 'Palayam, Kozhikode - Kerala PCB'},
 {'Unnamed: 0': 117,
  'StationId': 'KL007',
  'StationName': 'Kariavattom, Thiruvananthapuram - Kerala PCB'},
 {'Unnamed: 0': 118,
  'StationId': 'KL008',
  'StationName': 'Plammoodu, Thiruvananthapuram - Kerala PCB'},
 {'Unnamed: 0': 119,
  'StationId': 'MP001',
  'StationName': 'T T Nagar, Bhopal - MPPCB'},
 {'Unnamed: 0': 120,
  'StationId': 'MP002',
  'StationName': 'Shrivastav Colony, Damoh - MPPCB'},
 {'Unnamed: 0': 121,
  'StationId': 'MP003',
  'StationName': 'Bhopal Chauraha, Dewas - MPPCB'},
 {'Unnamed: 0': 122,
  'StationId': 'MP004',
  'StationName': 'City Center, Gwalior - MPPCB'},
 {'Unnamed: 0': 123,
  'StationId': 'MP005',
  'StationName': 'Phool Bagh, Gwalior - Mondelez Ind. Food'},
 {'Unnamed: 0': 124,
  'StationId': 'MP006',
  'StationName': 'Chhoti Gwaltoli, Indore - MPPCB'},
 {'Unnamed: 0': 125,
  'StationId': 'MP007',
  'StationName': 'Marhatal, Jabalpur - MPPCB'},
 {'Unnamed: 0': 126,
  'StationId': 'MP008',
  'StationName': 'Gole Bazar, Katni - MPPCB'},
 {'Unnamed: 0': 127,
  'StationId': 'MP009',
  'StationName': 'Sahilara, Maihar - KJS Cements'},
 {'Unnamed: 0': 128,
  'StationId': 'MP010',
  'StationName': 'Sector-D Industrial Area, Mandideep - MPPCB'},
 {'Unnamed: 0': 129,
  'StationId': 'MP011',
  'StationName': 'Sector-2 Industrial Area, Pithampur - MPPCB'},
 {'Unnamed: 0': 130,
  'StationId': 'MP012',
  'StationName': 'Shasthri Nagar, Ratlam - IPCA Lab'},
 {'Unnamed: 0': 131,
  'StationId': 'MP013',
  'StationName': 'Deen Dayal Nagar, Sagar - MPPCB'},
 {'Unnamed: 0': 132,
  'StationId': 'MP014',
  'StationName': 'Bandhavgar Colony, Satna - Birla Cement'},
 {'Unnamed: 0': 133,
  'StationId': 'MP015',
  'StationName': 'Vindhyachal STPS, Singrauli - MPPCB'},
 {'Unnamed: 0': 134,
  'StationId': 'MP016',
  'StationName': 'Mahakaleshwar Temple, Ujjain - MPPCB'},
 {'Unnamed: 0': 135,
  'StationId': 'MH001',
  'StationName': 'More Chowk Waluj, Aurangabad - MPCB'},
 {'Unnamed: 0': 136,
  'StationId': 'MH002',
  'StationName': 'Chandrapur, Chandrapur - MPCB'},
 {'Unnamed: 0': 137,
  'StationId': 'MH003',
  'StationName': 'MIDC Khutala, Chandrapur - MPCB'},
 {'Unnamed: 0': 138,
  'StationId': 'MH004',
  'StationName': 'Khadakpada, Kalyan - MPCB'},
 {'Unnamed: 0': 139,
  'StationId': 'MH005',
  'StationName': 'Bandra, Mumbai - MPCB'},
 {'Unnamed: 0': 140,
  'StationId': 'MH006',
  'StationName': 'Borivali East, Mumbai - MPCB'},
 {'Unnamed: 0': 141,
  'StationId': 'MH007',
  'StationName': 'Chhatrapati Shivaji Intl. Airport (T2), Mumbai - MPCB'},
 {'Unnamed: 0': 142,
  'StationId': 'MH008',
  'StationName': 'Colaba, Mumbai - MPCB'},
 {'Unnamed: 0': 143,
  'StationId': 'MH009',
  'StationName': 'Kurla, Mumbai - MPCB'},
 {'Unnamed: 0': 144,
  'StationId': 'MH010',
  'StationName': 'Powai, Mumbai - MPCB'},
 {'Unnamed: 0': 145,
  'StationId': 'MH011',
  'StationName': 'Sion, Mumbai - MPCB'},
 {'Unnamed: 0': 146,
  'StationId': 'MH012',
  'StationName': 'Vasai West, Mumbai - MPCB'},
 {'Unnamed: 0': 147,
  'StationId': 'MH013',
  'StationName': 'Vile Parle West, Mumbai - MPCB'},
 {'Unnamed: 0': 148,
  'StationId': 'MH014',
  'StationName': 'Worli, Mumbai - MPCB'},
 {'Unnamed: 0': 149,
  'StationId': 'MH015',
  'StationName': 'Opp GPO Civil Lines, Nagpur - MPCB'},
 {'Unnamed: 0': 150,
  'StationId': 'MH016',
  'StationName': 'Gangapur Road, Nashik - MPCB'},
 {'Unnamed: 0': 151,
  'StationId': 'MH017',
  'StationName': 'Airoli, Navi Mumbai - MPCB'},
 {'Unnamed: 0': 152,
  'StationId': 'MH018',
  'StationName': 'Mahape, Navi Mumbai - MPCB'},
 {'Unnamed: 0': 153,
  'StationId': 'MH019',
  'StationName': 'Nerul, Navi Mumbai - MPCB'},
 {'Unnamed: 0': 154,
  'StationId': 'MH020',
  'StationName': 'Karve Road, Pune - MPCB'},
 {'Unnamed: 0': 155,
  'StationId': 'MH021',
  'StationName': 'Solapur, Solapur - MPCB'},
 {'Unnamed: 0': 156,
  'StationId': 'MH022',
  'StationName': 'Pimpleshwar Mandir, Thane - MPCB'},
 {'Unnamed: 0': 157,
  'StationId': 'ML001',
  'StationName': 'Lumpyngngad, Shillong - Meghalaya PCB'},
 {'Unnamed: 0': 158,
  'StationId': 'MZ001',
  'StationName': 'Sikulpuikawn, Aizawl - Mizoram PCB'},
 {'Unnamed: 0': 159,
  'StationId': 'OD001',
  'StationName': 'GM Office, Brajrajnagar - OSPCB'},
 {'Unnamed: 0': 160,
  'StationId': 'OD002',
  'StationName': 'Talcher Coalfields,Talcher - OSPCB'},
 {'Unnamed: 0': 161,
  'StationId': 'PB001',
  'StationName': 'Golden Temple, Amritsar - PPCB'},
 {'Unnamed: 0': 162,
  'StationId': 'PB002',
  'StationName': 'Hardev Nagar, Bathinda - PPCB'},
 {'Unnamed: 0': 163,
  'StationId': 'PB003',
  'StationName': 'Civil Line, Jalandhar - PPCB'},
 {'Unnamed: 0': 164,
  'StationId': 'PB004',
  'StationName': 'Kalal Majra, Khanna - PPCB'},
 {'Unnamed: 0': 165,
  'StationId': 'PB005',
  'StationName': 'Punjab Agricultural University, Ludhiana - PPCB'},
 {'Unnamed: 0': 166,
  'StationId': 'PB006',
  'StationName': 'RIMT University, Mandi Gobindgarh - PPCB'},
 {'Unnamed: 0': 167,
  'StationId': 'PB007',
  'StationName': 'Model Town, Patiala - PPCB'},
 {'Unnamed: 0': 168,
  'StationId': 'PB008',
  'StationName': 'Ratanpura, Rupnagar - Ambuja Cements'},
 {'Unnamed: 0': 169,
  'StationId': 'RJ001',
  'StationName': 'Moti Doongri, Alwar - RSPCB'},
 {'Unnamed: 0': 170,
  'StationId': 'RJ002',
  'StationName': 'Civil Lines, Ajmer - RSPCB'},
 {'Unnamed: 0': 171,
  'StationId': 'RJ003',
  'StationName': 'RIICO Ind. Area III, Bhiwadi - RSPCB'},
 {'Unnamed: 0': 172,
  'StationId': 'RJ004',
  'StationName': 'Adarsh Nagar, Jaipur - RSPCB'},
 {'Unnamed: 0': 173,
  'StationId': 'RJ005',
  'StationName': 'Police Commissionerate, Jaipur - RSPCB'},
 {'Unnamed: 0': 174,
  'StationId': 'RJ006',
  'StationName': 'Shastri Nagar, Jaipur - RSPCB'},
 {'Unnamed: 0': 175,
  'StationId': 'RJ007',
  'StationName': 'Collectorate, Jodhpur - RSPCB'},
 {'Unnamed: 0': 176,
  'StationId': 'RJ008',
  'StationName': 'Shrinath Puram, Kota - RSPCB'},
 {'Unnamed: 0': 177,
  'StationId': 'RJ009',
  'StationName': 'Indira Colony Vistar, Pali - RSPCB'},
 {'Unnamed: 0': 178,
  'StationId': 'RJ010',
  'StationName': 'Ashok Nagar, Udaipur - RSPCB'},
 {'Unnamed: 0': 179,
  'StationId': 'TN001',
  'StationName': 'Alandur Bus Depot, Chennai - CPCB'},
 {'Unnamed: 0': 180,
  'StationId': 'TN002',
  'StationName': 'Manali Village, Chennai - TNPCB'},
 {'Unnamed: 0': 181,
  'StationId': 'TN003',
  'StationName': 'Manali, Chennai - CPCB'},
 {'Unnamed: 0': 182,
  'StationId': 'TN004',
  'StationName': 'Velachery Res. Area, Chennai - CPCB'},
 {'Unnamed: 0': 183,
  'StationId': 'TN005',
  'StationName': 'SIDCO Kurichi, Coimbatore - TNPCB'},
 {'Unnamed: 0': 184,
  'StationId': 'TG001',
  'StationName': 'Bollaram Industrial Area, Hyderabad - TSPCB'},
 {'Unnamed: 0': 185,
  'StationId': 'TG002',
  'StationName': 'Central University, Hyderabad - TSPCB'},
 {'Unnamed: 0': 186,
  'StationId': 'TG003',
  'StationName': 'ICRISAT Patancheru, Hyderabad - TSPCB'},
 {'Unnamed: 0': 187,
  'StationId': 'TG004',
  'StationName': 'IDA Pashamylaram, Hyderabad - TSPCB'},
 {'Unnamed: 0': 188,
  'StationId': 'TG005',
  'StationName': 'Sanathnagar, Hyderabad - TSPCB'},
 {'Unnamed: 0': 189,
  'StationId': 'TG006',
  'StationName': 'Zoo Park, Hyderabad - TSPCB'},
 {'Unnamed: 0': 190,
  'StationId': 'UP001',
  'StationName': 'Sanjay Palace, Agra - UPPCB'},
 {'Unnamed: 0': 191,
  'StationId': 'UP002',
  'StationName': 'New Collectorate, Baghpat - UPPCB'},
 {'Unnamed: 0': 192,
  'StationId': 'UP003',
  'StationName': 'Yamunapuram, Bulandshahr - UPPCB'},
 {'Unnamed: 0': 193,
  'StationId': 'UP004',
  'StationName': 'Indirapuram, Ghaziabad - UPPCB'},
 {'Unnamed: 0': 194,
  'StationId': 'UP005',
  'StationName': 'Loni, Ghaziabad - UPPCB'},
 {'Unnamed: 0': 195,
  'StationId': 'UP006',
  'StationName': 'Sanjay Nagar, Ghaziabad - UPPCB'},
 {'Unnamed: 0': 196,
  'StationId': 'UP007',
  'StationName': 'Vasundhara, Ghaziabad - UPPCB'},
 {'Unnamed: 0': 197,
  'StationId': 'UP008',
  'StationName': 'Knowledge Park - III, Greater Noida - UPPCB'},
 {'Unnamed: 0': 198,
  'StationId': 'UP009',
  'StationName': 'Knowledge Park - V, Greater Noida - UPPCB'},
 {'Unnamed: 0': 199,
  'StationId': 'UP010',
  'StationName': 'Anand Vihar, Hapur - UPPCB'},
 {'Unnamed: 0': 200,
  'StationId': 'UP011',
  'StationName': 'Nehru Nagar, Kanpur - UPPCB'},
 {'Unnamed: 0': 201,
  'StationId': 'UP012',
  'StationName': 'Central School, Lucknow - CPCB'},
 {'Unnamed: 0': 202,
  'StationId': 'UP013',
  'StationName': 'Gomti Nagar, Lucknow - UPPCB'},
 {'Unnamed: 0': 203,
  'StationId': 'UP014',
  'StationName': 'Lalbagh, Lucknow - CPCB'},
 {'Unnamed: 0': 204,
  'StationId': 'UP015',
  'StationName': 'Nishant Ganj, Lucknow - UPPCB'},
 {'Unnamed: 0': 205,
  'StationId': 'UP016',
  'StationName': 'Talkatora District Industries Center, Lucknow - CPCB'},
 {'Unnamed: 0': 206,
  'StationId': 'UP017',
  'StationName': 'Ganga Nagar, Meerut - UPPCB'},
 {'Unnamed: 0': 207,
  'StationId': 'UP018',
  'StationName': 'Jai Bhim Nagar, Meerut - UPPCB'},
 {'Unnamed: 0': 208,
  'StationId': 'UP019',
  'StationName': 'Pallavpuram Phase 2, Meerut - UPPCB'},
 {'Unnamed: 0': 209,
  'StationId': 'UP020',
  'StationName': 'Lajpat Nagar, Moradabad - UPPCB'},
 {'Unnamed: 0': 210,
  'StationId': 'UP021',
  'StationName': 'New Mandi, Muzaffarnagar - UPPCB'},
 {'Unnamed: 0': 211,
  'StationId': 'UP022',
  'StationName': 'Sector - 125, Noida - UPPCB'},
 {'Unnamed: 0': 212,
  'StationId': 'UP023',
  'StationName': 'Sector - 62, Noida - IMD'},
 {'Unnamed: 0': 213,
  'StationId': 'UP024',
  'StationName': 'Sector-1, Noida - UPPCB'},
 {'Unnamed: 0': 214,
  'StationId': 'UP025',
  'StationName': 'Sector-116, Noida - UPPCB'},
 {'Unnamed: 0': 215,
  'StationId': 'UP026',
  'StationName': 'Ardhali Bazar, Varanasi - UPPCB'},
 {'Unnamed: 0': 216,
  'StationId': 'WB001',
  'StationName': 'Asansol Court Area, Asansol - WBPCB'},
 {'Unnamed: 0': 217,
  'StationId': 'WB002',
  'StationName': 'Sidhu Kanhu Indoor Stadium, Durgapur - WBPCB'},
 {'Unnamed: 0': 218,
  'StationId': 'WB003',
  'StationName': 'Haldia, Haldia - WBPCB'},
 {'Unnamed: 0': 219,
  'StationId': 'WB004',
  'StationName': 'Belur Math, Howrah - WBPCB'},
 {'Unnamed: 0': 220,
  'StationId': 'WB005',
  'StationName': 'Ghusuri, Howrah - WBPCB'},
 {'Unnamed: 0': 221,
  'StationId': 'WB006',
  'StationName': 'Padmapukur, Howrah - WBPCB'},
 {'Unnamed: 0': 222,
  'StationId': 'WB007',
  'StationName': 'Ballygunge, Kolkata - WBPCB'},
 {'Unnamed: 0': 223,
  'StationId': 'WB008',
  'StationName': 'Bidhannagar, Kolkata - WBPCB'},
 {'Unnamed: 0': 224,
  'StationId': 'WB009',
  'StationName': 'Fort William, Kolkata - WBPCB'},
 {'Unnamed: 0': 225,
  'StationId': 'WB010',
  'StationName': 'Jadavpur, Kolkata - WBPCB'},
 {'Unnamed: 0': 226,
  'StationId': 'WB011',
  'StationName': 'Rabindra Bharati University, Kolkata - WBPCB'},
 {'Unnamed: 0': 227,
  'StationId': 'WB012',
  'StationName': 'Rabindra Sarobar, Kolkata - WBPCB'},
 {'Unnamed: 0': 228,
  'StationId': 'WB013',
  'StationName': 'Victoria, Kolkata - WBPCB'},
 {'Unnamed: 0': 229,
  'StationId': 'WB014',
  'StationName': 'Ward-32 Bapupara, Siliguri - WBPCB'}]

for station in stations:
    # Split the StationName at ' - ' and take the first part
    station['StationName'] = station['StationName'].split(' - ')[0]

# Print the modified list
print(stations)


[{'Unnamed: 0': 0, 'StationId': 'AP001', 'StationName': 'Secretariat, Amaravati'}, {'Unnamed: 0': 1, 'StationId': 'AP002', 'StationName': 'Anand Kala Kshetram, Rajamahendravaram'}, {'Unnamed: 0': 2, 'StationId': 'AP003', 'StationName': 'Tirumala, Tirupati'}, {'Unnamed: 0': 3, 'StationId': 'AP004', 'StationName': 'PWD Grounds, Vijayawada'}, {'Unnamed: 0': 4, 'StationId': 'AP005', 'StationName': 'GVM Corporation, Visakhapatnam'}, {'Unnamed: 0': 5, 'StationId': 'AS001', 'StationName': 'Railway Colony, Guwahati'}, {'Unnamed: 0': 6, 'StationId': 'BR001', 'StationName': 'Collectorate, Gaya'}, {'Unnamed: 0': 7, 'StationId': 'BR002', 'StationName': 'SFTI Kusdihra, Gaya'}, {'Unnamed: 0': 8, 'StationId': 'BR003', 'StationName': 'Industrial Area, Hajipur'}, {'Unnamed: 0': 9, 'StationId': 'BR004', 'StationName': 'Muzaffarpur Collectorate, Muzaffarpur'}, {'Unnamed: 0': 10, 'StationId': 'BR005', 'StationName': 'DRM Office Danapur, Patna'}, {'Unnamed: 0': 11, 'StationId': 'BR006', 'StationName': 'Gov

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)



import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

x_train['StationId'] = label_encoder.fit_transform(x_train['StationId'])

x_test['StationId'] = label_encoder.fit_transform(x_test['StationId'])


In [ ]:
x_train['Date'] = pd.to_datetime(x_train['Date'], errors='coerce')  # Ensure proper conversion


x_train['year'] = x_train['Date'].dt.year
x_train['month'] = x_train['Date'].dt.month
x_train['day'] = x_train['Date'].dt.day
x_train['hour'] = x_train['Date'].dt.hour
x_train['day_of_week'] = x_train['Date'].dt.dayofweek  # Monday = 0, Sunday = 6
x_train['is_weekend'] = x_train['day_of_week'] >= 5  # Boolean column for weekends (Saturday/Sunday)

# Step 3: Drop the original 'Datetime' column if no longer needed
x_train = x_train.drop('Date', axis=1)

x_train

In [114]:
x_test['Date'] = pd.to_datetime(x_test['Date'], errors='coerce')  # Ensure proper conversion


x_test['year'] = x_test['Date'].dt.year
x_test['month'] = x_test['Date'].dt.month
x_test['day'] = x_test['Date'].dt.day
x_test['hour'] = x_test['Date'].dt.hour
x_test['day_of_week'] = x_test['Date'].dt.dayofweek  # Monday = 0, Sunday = 6
x_test['is_weekend'] = x_test['day_of_week'] >= 5  # Boolean column for weekends (Saturday/Sunday)

# Step 3: Drop the original 'Datetime' column if no longer needed
x_test = x_test.drop('Date', axis=1)

x_test

,StationId,year,month,day,hour,day_of_week,is_weekend
107783,109,2019,10,24,0,3,False
59608,55,2018,12,8,0,5,True
23036,23,2017,1,22,0,6,True
100858,99,2019,11,7,0,3,False
91928,92,2020,5,12,0,1,False
...,...,...,...,...,...,...,...
99032,98,2015,9,19,0,5,True
74770,73,2020,6,15,0,0,False
45608,42,2017,12,9,0,5,True
82055,85,2019,12,25,0,2,False


In [115]:
x_train = np.expand_dims(x_train, axis=2) 
x_test = np.expand_dims(x_test, axis=2)

In [116]:
model = Sequential()

# LSTM layer with 64 units, input shape as (32, 1)
model.add(LSTM(64, return_sequences=True, input_shape=(32, 1)))
model.add(Dropout(0.2))  # Regularization to prevent overfitting

# LSTM layer with 64 units
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))

# Dense layer with 128 units and ReLU activation
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))

# Dense layer with 64 units and ReLU activation
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))

# Output layer for regression with 13 units (one for each target variable)
model.add(Dense(13))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [117]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [118]:
model.compile(optimizer="adam",metrics=["mean_squared_error"],loss="mean_absolute_error")

In [119]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(86428, 7, 1)
(86428, 13)
(21607, 7, 1)
(21607, 13)


In [120]:
model.fit(x_train,y_train,epochs=50,validation_data=[x_test,y_test])

Epoch 1/50
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 25.9683 - mean_squared_error: 3189.5142 - val_loss: 20.0773 - val_mean_squared_error: 1964.6959
Epoch 2/50
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 21.1912 - mean_squared_error: 2125.2917 - val_loss: 19.8493 - val_mean_squared_error: 1974.9023
Epoch 3/50
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 20.8524 - mean_squared_error: 2061.3281 - val_loss: 19.7668 - val_mean_squared_error: 1958.0464
Epoch 4/50
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 20.5965 - mean_squared_error: 2020.9159 - val_loss: 19.7481 - val_mean_squared_error: 1954.6099
Epoch 5/50
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 20.3138 - mean_squared_error: 1960.4441 - val_loss: 19.3633 - val_mean_squared_error: 1848.5535
Epoch 6/50
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 20.2409 - mean_squared_error: 1965.2258 - val_loss: 19.1913 - val_mean_squared_error: 1865.7134
Epoch 7/50
2701/2701 ━━━━━━━━━━━━━━━━━━━━ 17s 

In [129]:
single_sample = np.reshape(x_test[1], (1, 7, 1, 1))  


prediction = model.predict(single_sample)


print("Prediction for single test sample:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Prediction for single test sample: [[ 53.102      154.98346     10.074654    26.208534    24.621841
   28.619661     0.89111173   7.9386563   36.026627     1.9172943
    6.446809     2.432784   119.633026  ]]


In [130]:
y_test.iloc[1]

PM2.5       37.320000
PM10       157.968430
NO           5.540000
NO2         19.940001
NOx         25.480000
NH3         28.732876
CO           0.850000
SO2          3.980000
O3          20.270000
Benzene      0.600000
Toluene      0.640000
Xylene       2.423446
AQI         70.000000
Name: 59608, dtype: float32

In [128]:
import pickle

# Save model or any object
with open('station.pkl', 'wb') as file:
    pickle.dump(model, file)